In [1]:
from hathor import Hathor
import sys
import os
hathor_root = os.path.dirname(os.getcwd())
sys.path.insert(0, hathor_root)
from inference_auth_token import get_access_token
access_token = get_access_token()
config_list = [{
    "model": "openai/gpt-oss-120b",  # or whatever model name your endpoint expects
    "api_key": access_token,
    "base_url": "https://inference-api.alcf.anl.gov/resource_server/sophia/vllm/v1",
    "api_type": "openai",
    "use_functions_api": True,
    "tools": [],
}]
hathor = Hathor(config_list=config_list)
hathor.run()

/opt/miniconda3/envs/globus_env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


ValidationError: 1 validation error for _LLMConfig
config_list.1.openai.model
  Field required [type=missing, input_value={'api_type': 'openai', 't...se_functions_api': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

**Hypothesis**

*“The onset of rapid, bursty star formation in low‑mass (M\*_halo ≲ 10¹⁰ M⊙) dwarf galaxies at redshift z ≈ 2–4 is primarily regulated by the timing and strength of early, metal‑poor supernova‑driven outflows, rather than by cosmological gas accretion or re‑ionisation heating.”*

**Rationale**

1. **Theoretical background** – In ΛCDM, dwarf haloes grow mainly by smooth accretion and minor mergers. Classical models attribute their star‑formation histories (SFHs) to external heating (UV background) and internal feedback. Recent high‑resolution simulations suggest that *early* super‑nova (SN) explosions can evacuate gas, creating a “breathing” cycle that drives bursty SF. Distinguishing whether this internal regulation dominates over external heating is still an open question.

2. **Observational motivation** – Deep HST imaging of faint dwarfs at z ≈ 2–4 shows episodic SF with duty cycles of a few hundred Myr, while their UV luminosities imply high specific SFRs. Spectroscopic metallicities are low (Z ≲ 0.1 Z⊙), consistent with metal‑poor outflows.

3. **Testability with RAMSES** – The RAMSES AMR code includes:
   - **Metal‑dependent cooling** and a **UV background** (e.g. Haardt‑Madau).
   - **Stellar feedback** modules (thermal + kinetic SN, radiation pressure).
   - **Tracer particles / passive scalars** for metal enrichment.
   - **Halo catalogs** (via HOP or Rockstar) and merger trees.

These ingredients allow us to isolate the impact of SN‑driven outflows by comparing runs with identical cosmology and UV background but different SN feedback prescriptions (e.g., “standard” vs. “enhanced” momentum injection).

---

**Plot Idea**

*“Burst‑Cycle Phase Diagram: Specific Star‑Formation Rate (sSFR) vs. Central Gas‑Mass Fraction (f_gas, r < 0.1 R_vir) for dwarf haloes, colour‑coded by outflow metallicity, with over‑plotted tracks for individual haloes across cosmic time (z = 4 → 2).”*

**What the Plot Shows**

- **X‑axis (f_gas)** – Fraction of baryons retained in the inner 10 % of the virial radius. Low values indicate that a recent outflow has cleared the central reservoir.
- **Y‑axis (sSFR)** – Instantaneous specific SFR (M⊙ yr⁻¹ / M\*_star). High sSFR together with low f_gas signals a *burst* triggered by fresh inflow after an outflow.
- **Colour (Z_out)** – Mass‑weighted metallicity of gas that has left the halo in the last 50 Myr (traced by passive scalars). Metal‑poor outflows (blue) imply early SN‑driven winds; metal‑rich outflows (red) would suggest recycling of enriched gas.
- **Tracks** – Each dwarf halo is plotted as a line connecting its successive snapshots (Δt ≈ 20 Myr). The direction of the arrow indicates the temporal evolution.

**Why This Plot Tests the Hypothesis**

- If **SN‑driven outflows dominate**, we expect a *tight anti‑correlation* between f_gas and sSFR, with bursts occurring shortly after the central gas fraction recovers. The outflow metallicity should remain low (blue) during the first few bursts, rising only after several cycles of enrichment.
- If **external heating or accretion dominates**, the relation will be weaker, and outflow metallicities will be higher (red) because the gas being expelled is already enriched by prior star formation.

**Implementation Details (RAMSES‑friendly)**

| Step | Action | RAMSES Feature / Tool |
|------|--------|-----------------------|
| 1 | **Select dwarf haloes** (M_halo ≤ 10¹⁰ M⊙) from the halo catalog at z = 4. | Rockstar/HOP output; use `pynbody` or `yt` to read. |
| 2 | **Extract snapshots** every ~20 Myr from z = 4 to 2 (≈ 30 snapshots). | RAMSES outputs (`output_XXXXX/`). |
| 3 | **Compute central gas mass** within 0.1 R_vir. | `yt` region selection + `sum('gas', 'cell_mass')`. |
| 4 | **Calculate instantaneous SFR** from star‑particle formation rates over the last Δt. | Sum of `star_particle` masses formed in Δt / Δt. |
| 5 | **Track outflowing gas** using a passive scalar that tags gas crossing a spherical shell at 0.9 R_vir with outward velocity > v_esc. Record its metallicity. | Add a scalar field in RAMSES (`outflow_tag`) and post‑process with `yt`. |
| 6 | **Assemble the phase‑space data** for each halo and each snapshot. | Store in an HDF5 table (≈ few MB per halo). |
| 7 | **Plot** using `matplotlib`: scatter points with arrows, colour map for metallicity, and optional density contours. | `plt.scatter`, `plt.quiver` for arrows. |
| 8 | **Statistical test** – compute Pearson/Spearman correlation between f_gas and sSFR for each feedback model; compare distributions of Z_out. | `scipy.stats`. |

**Computational Feasibility**

- **Memory** – Each snapshot for a (∼ 10 Mpc) box at ∼ 10 pc resolution is ≈ 2 GB. By loading only the cells belonging to selected haloes (via a mask) the RAM usage stays < 4 GB.
- **CPU** – Computing the central gas mass and SFR for ~200 dwarfs across 30 snapshots takes ≈ 2 h on a 6‑core laptop (using `yt` with parallelism).
- **Disk** – The final compiled dataset (halo ID, redshift, f_gas, sSFR, Z_out) is < 10 MB, easily handled.

**Potential Extensions (if time permits)**

- Overlay the **UV background heating rate** (from the simulation’s cooling table) as a secondary colour axis to verify that it remains sub‑dominant.
- Add a **control sample** of identical haloes from a run with *no* SN feedback to highlight the role of outflows.

---

**Final Recommendation**

Adopt the **burst‑cycle phase diagram** as the primary diagnostic plot to evaluate the hypothesis that early, metal‑poor supernova‑driven outflows are the chief regulator of bursty star formation in low‑mass dwarf galaxies at z ≈ 2–4. This approach leverages RAMSES’s strengths (AMR resolution, tracer fields, built‑in feedback modules) while remaining tractable on a research‑grade laptop.

**TERMINATE**

In [ ]:
import arxiv
import numpy as np

import requests
from io import BytesIO
import fitz

In [18]:
client = arxiv.Client()
search =arxiv.Search(query="circumgalactic medium",  
                        max_results=20,
                        sort_by=arxiv.SortCriterion.SubmittedDate,
                        sort_order=arxiv.SortOrder.Descending 
                    )
results = list(client.results(search))

chosen_papers = np.random.choice(results, size=5, replace=False)

search =arxiv.Search(query="circumgalactic medium",  
                        max_results=20,
                        sort_by=arxiv.SortCriterion.Relevance,
                        sort_order=arxiv.SortOrder.Descending 
                    )
results = list(client.results(search))
relevant_papers = np.random.choice(results, size=5, replace=False)

chosen_papers = np.concatenate([chosen_papers,relevant_papers])

for paper in chosen_papers:
    # Download and extract full text
    try:
        response = requests.get(paper.pdf_url)
        pdf_file = BytesIO(response.content)
        
        doc = fitz.open(stream=pdf_file, filetype="pdf")
        full_text = ""
        for page in doc:
            full_text += page.get_text()
        doc.close()
    except Exception as e:
        full_text = f"[Error extracting text: {e}]"
    
    # Create paper string with correct attributes
    paper_string = f"""
                    Title: {paper.title}
                    Abstract: {paper.summary}
                    Full Text: {full_text}
                    """
    print(paper_string)
 



                    Title: Phononic Casimir Effect in Planar Materials
                    Abstract: The Phononic Casimir effect between planar objects is investigated by deriving a formalism from the quantum partition function of the system following multiscattering approach. This fluctuation-induced coupling is mediated by phonons modeled as an effective elastic medium. We find that excitations with three types of polarizations arise from the resolved boundary conditions, however the coupling is dominated by only one of these degrees of freedom due to exponential suppression effects in the other two. The obtained scaling laws and dependence on materials properties and temperature suggest effective pathways of interaction control. Scenarios of materials combinations are envisioned where the Phononic Casimir effect is of similar order as the standard Casimir interaction mediated by electromagnetic fluctuations.
                    Full Text: [Error extracting text: name 'requests' is 